In [2]:
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
import itertools

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
from torchvision import transforms
import copy

import random
import time

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
# Colab specific cells

# torch.manual_seed(42)

# Get sample image from Google Drive and load it for debugging. Will need to
# modify code to deal with mini-batch of RGB image.
#from google.colab import drive
#drive.mount('/gdrive', force_remount=True)
# In Google Drive, create a shortcut link of the shared Project folder so that it appears in your root Google Drive directory
# i.e., Right Click Project folder: "Add Shortcut to Drive"

#%cd '/gdrive/MyDrive/Project/'
#%ls


In [18]:
# Data should be in a dir named 'data/'
data = np.load('data/RESISC45_images.npy')
labels = np.load('data/RESISC45_classes.npy')
classes = np.load('data/RESISC45_class_names.npy')

print('Training data shape: ', data.shape)
print('Testing data shape: ', labels.shape)
print('Num Classes', classes.shape)

img_size = 256

Training data shape:  (31500, 256, 256, 3)
Testing data shape:  (31500,)
Num Classes (45,)


In [24]:

# Create full-factorial combination of the augmentations. Each
# one will be a "task." Each augmentation will only have 2 levels (on and off).
# Hence, there will be 512 tasks if there are 9 augs with 2 levels each.

# First, set number of augmentations per task.
numAugs = 7
augTasks = list(itertools.product([0, 1], repeat=numAugs))

# Please note that augTasks is in counting order: we may have to sample from it later
#print(augTasks)
print(len(augTasks))

transform_list = [transforms.RandomHorizontalFlip(), 
            transforms.RandomVerticalFlip(1.0),
            transforms.RandomHorizontalFlip(1.0),
            transforms.RandomPerspective(distortion_scale=0.1, p=1.0, fill=0.5),
            transforms.RandomResizedCrop(img_size // 1.08),
            transforms.ColorJitter(saturation=4.0, hue=0.01),
            transforms.ColorJitter(brightness=0.5, contrast=0.9)]

def task_batch_transformer(batch, task_list):
    # Task list is a list of lists: num_tasks, task_inds
    batch_list = []
    for task_vec in task_list:
        tbatch = batch.copy()
        
        # For each index of the single task-vector ([0,0,1,0,1,0,0] for example), do the index transform 
        for task_i in range(len(task_vec)):
            if task_vec[task_i] == 1:
                tbatch = transform_list[task_i](tbatch)
                
        batch_list.append(tbatch)
        
    return batch_list, task_list # List of image batches and their corresponding tasks

128


In [37]:
# Demonstration
data_batch = data[0:8]

task_batch = [[1,0,0,0,0,0,1], [0,1,1,0,0,0,0]]

out = task_batch_transformer(data_batch, task_batch)

(8, 256, 256, 3)
0 255
<class 'numpy.ndarray'>
(8, 3, 256, 256)


TypeError: Cannot handle this data type: (1, 1, 256, 256), |u1

In [ ]:
 # Below here is old code

In [8]:

class addGaussianNoise(object):
    # Transform to add gaussian noise since PyTorch did not have one (that I know of).
    def __init__(self, mean=0.0, std=1.0, p=0.5):
        self.mean = mean
        self.std = std
        self.p = p
      
    def __call__(self, img):
        if torch.rand(1).item() < self.p:
            return img + torch.randn(img.size()) * self.std + self.mean
        return img
        
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1}, p={2})'.format(self.mean, self.std, self.p)



In [9]:

def composeAugs(params):
    # Function to compose the augmentations, the order of which will be fixed.
    # Some of these parameters are best guess and can be changed if we need to.
    # For example, hue is set to have very very minor change because it can 
    # create false colors (green grass can become magenta). If the team decides
    # that false colors are acceptable, we can change the hue parameter.
    
    # Scripting the transformations will not work here due to the inclusion of
    # addGaussianNoise transform. Use transform composition instead.
    


    pHF, pVF, rotAng, pPersp, cropScale, pNoise, satVal, brightVal, contrastVal, blurSigma = params

    imXforms = transforms.Compose([
        transforms.RandomHorizontalFlip(p=pHF),
        transforms.RandomVerticalFlip(p=pVF),
        transforms.RandomRotation(rotAng, fill=0.5),
        transforms.RandomPerspective(distortion_scale=0.1, p=pPersp, fill=0.5),
        transforms.RandomResizedCrop(256,
                                    scale=(cropScale, 1.0),
                                    ratio=(1.0, 1.0),
                                    interpolation=transforms.InterpolationMode.BILINEAR),
        addGaussianNoise(std=0.1, p=pNoise),
        transforms.ColorJitter(saturation=satVal, hue=0.01),
        transforms.ColorJitter(brightness=brightVal, contrast=contrastVal),
        transforms.GaussianBlur(9, sigma=blurSigma)
        ])
        
    return imXforms



In [10]:

# set augmentation parameters for input task
def makeAugParams(task):
    pHF = 0.99 if task[0] == 1 else 0.0
    pVF = 0.99 if task[1] == 1 else 0.0
    rotAng = 359.0 if task[2] == 1 else 0.0
    pPersp = 0.99 if task[3] == 1 else 0.0
    cropScale = 0.5 if task[4] == 1 else 1.0
    pNoise = 0.99 if task[5] == 1 else 0.0
    satVal = 4.0 if task[6] == 1 else 0.0
    brightVal = 0.5 if task[7] == 1 else 0.0
    contrastVal = 0.9 if task[7] == 1 else 0.0
    blurSigma = (0.01, 2.0) if task[8] == 1 else 1e-9

    return (pHF, pVF, rotAng, pPersp, cropScale, pNoise, 
            satVal, brightVal, contrastVal, blurSigma)
    

[(0, 0, 0, 0, 0, 0, 0, 0, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 1, 1), (0, 0, 0, 0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 1), (0, 0, 0, 0, 0, 0, 1, 1, 0), (0, 0, 0, 0, 0, 0, 1, 1, 1), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 1), (0, 0, 0, 0, 0, 1, 0, 1, 0), (0, 0, 0, 0, 0, 1, 0, 1, 1), (0, 0, 0, 0, 0, 1, 1, 0, 0), (0, 0, 0, 0, 0, 1, 1, 0, 1), (0, 0, 0, 0, 0, 1, 1, 1, 0), (0, 0, 0, 0, 0, 1, 1, 1, 1), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 1), (0, 0, 0, 0, 1, 0, 0, 1, 0), (0, 0, 0, 0, 1, 0, 0, 1, 1), (0, 0, 0, 0, 1, 0, 1, 0, 0), (0, 0, 0, 0, 1, 0, 1, 0, 1), (0, 0, 0, 0, 1, 0, 1, 1, 0), (0, 0, 0, 0, 1, 0, 1, 1, 1), (0, 0, 0, 0, 1, 1, 0, 0, 0), (0, 0, 0, 0, 1, 1, 0, 0, 1), (0, 0, 0, 0, 1, 1, 0, 1, 0), (0, 0, 0, 0, 1, 1, 0, 1, 1), (0, 0, 0, 0, 1, 1, 1, 0, 0), (0, 0, 0, 0, 1, 1, 1, 0, 1), (0, 0, 0, 0, 1, 1, 1, 1, 0), (0, 0, 0, 0, 1, 1, 1, 1, 1), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 1), (0, 0, 0, 1, 

In [13]:

# select random task (for testing)
# in actual implementation we would iterate (sample?) task list # 
current_task = augTasks[random.randint(0,512)]


In [16]:

# define augmentation transforms
aug_params = makeAugParams(current_task)
image_transforms = composeAugs(aug_params)


In [17]:

# prepare a minibatch of images
batch_size = 256

d = data[0:batch_size]
d = d / 255.0
d = np.moveaxis(d, 3, 1)
d = torch.as_tensor(d)


In [18]:

# transform minibatch of images
t0 = time.time()
transformed_batch = image_transforms(d)
print("Time to transform batch of size {:d}: {:0f}".format(len(d), time.time()-t0))


Time to transform batch of size 256: 18.569443


In [ ]:

# plot sample of results
from mpl_toolkits.axes_grid1 import ImageGrid

transformed_batch = transformed_batch.numpy()
transformed_batch = np.moveaxis(transformed_batch, 1, 3)

fig = plt.figure(figsize=(15., 15.))
grid = ImageGrid(fig, 111,  
                 nrows_ncols=(4, 4),  
                 axes_pad=0.1)

for ax, im in zip(grid, transformed_batch[0:16]):
    # Iterating over the grid returns the Axes.
    ax.imshow(im)

plt.show()

del transformed_batch

